In [1]:
sc

In [4]:
ls -l

total 12
drwxrwxr-x 2 bjur bjur 4096 jun  7 17:20 Data/
-rw-rw-r-- 1 bjur bjur   10 jun  7 17:20 README.md
-rw-rw-r-- 1 bjur bjur   72 jun  7 17:21 Untitled.ipynb


In [1]:
Logs = sc.textFile('Data/advertising.csv')

In [2]:
Logs.take(5)

[u',pub_domain,user_id,viewed,ctr,click',
 u'1000,1000,1410160,True,0.463089335003,True',
 u'1000,1000,1157972,True,0.463089335003,False',
 u'1000,1000,1061623,True,0.463089335003,True',
 u'1001,1001,1169015,True,0.303265889759,True']

In [3]:
Lorgs_norm = Logs.map(lambda l: l.split(','))
#De momento me quedo con View = True xq para el reomendador son usuarios afines entre publishers, independientemente haya evento o no

In [4]:
Lorgs_norm.take(3)

[[u'', u'pub_domain', u'user_id', u'viewed', u'ctr', u'click'],
 [u'1000', u'1000', u'1410160', u'True', u'0.463089335003', u'True'],
 [u'1000', u'1000', u'1157972', u'True', u'0.463089335003', u'False']]

In [5]:
a = Lorgs_norm.map(lambda tokens: (tokens[1],tokens[2],tokens[3],tokens[4],tokens[5])).cache()

In [8]:
Log2Rec.take(2)

[(u'user_id', u'pub_domain', 0), (u'1410160', u'1000', 1)]

In [7]:
#Hago una tupla para mandarla al reccomendador,
#Primero User, luego Publisher y finalmente yes or no si hay CTR
Log2Rec = Lorgs_norm.map(lambda tokens: (tokens[2],tokens[1],1 if tokens[5] == "True" else 0)).cache()

In [9]:
#Divido en tres muestras (60,20,2) 
training_RDD, validation_RDD, test_RDD = Log2Rec.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [10]:
training_RDD.take(2)

[(u'1410160', u'1000', 1), (u'1157972', u'1000', 0)]

In [11]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.428348875339
For rank 8 the RMSE is 0.414329847038
For rank 12 the RMSE is 0.409517545949
The best model was trained with rank 12


In [21]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = 15
errors = [0, 0, 0]
err = 0
tolerance = 0.02
model2 = ALS.trainImplicit(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter, alpha= 0.1)

In [12]:
predictions.take(

[((1211416, 4754), -0.10082496701343394),
 ((1339720, 1780), 0.0),
 ((1293580, 2304), -0.17215800845031254)]

In [13]:
rates_and_preds.take(30)

[((1238875, 2839), (0.0, -0.06643802220337772)),
 ((1178711, 1253), (0.0, 0.0)),
 ((1299202, 3024), (0.0, 0.0)),
 ((1304318, 1346), (0.0, 0.0)),
 ((1260565, 2365), (0.0, 0.0)),
 ((1280198, 1694), (0.0, 0.0)),
 ((1379538, 2800), (0.0, 0.0)),
 ((1142153, 1971), (0.0, -0.15116990760060742)),
 ((1148723, 1451), (0.0, 0.17749708379423781)),
 ((1216253, 1245), (0.0, 0.0)),
 ((1379052, 2208), (0.0, 0.0)),
 ((1308003, 2967), (0.0, 0.0)),
 ((1144303, 2783), (0.0, -0.029833808085095337)),
 ((1424644, 1598), (0.0, 0.0)),
 ((1210293, 2441), (0.0, -0.19611470561792116)),
 ((1106123, 2113), (0.0, -0.1759580570662202)),
 ((1421026, 3224), (0.0, 0.0)),
 ((1213341, 2223), (0.0, 0.0)),
 ((1145716, 1538), (0.0, 0.0)),
 ((1234225, 2443), (0.0, 0.0)),
 ((1388419, 3285), (0.0, 0.0)),
 ((1172845, 2395), (0.0, 0.028677720069723252)),
 ((1263505, 2127), (0.0, -0.17685081008038478)),
 ((1094170, 1208), (1.0, 0.0)),
 ((1351991, 3435), (0.0, 0.0)),
 ((1324072, 1216), (0.0, 0.0)),
 ((1347376, 1536), (0.0, 0.0)),
 

In [96]:
a = model.recommendUsers(1216,20000)


In [99]:
a[1:3]

[Rating(user=1411035, product=1216, rating=0.8445205086801759),
 Rating(user=1260096, product=1216, rating=0.8445205086801759)]

In [102]:
from pyspark import Row
ratings = sc.parallelize(a)

In [106]:
df = ratings.toDF()

In [113]:
type(df)
df.count
df2 = df.filter('rating<0.5')
df2.count()

18966

In [97]:
def line2amountDest(l):
    fields = l.split(',')
    user = fields[0]
    product = fields[1]    
    rating = fields[2]
    
    return (user, product,rating)


In [63]:
Log2Rec.take(2)

[(u'user_id', u'pub_domain', 0), (u'1410160', u'1000', 1)]

In [25]:
Log2Rec_O = Log2Rec.map(lambda (x,y,z): (x,(y,z,1)))
Log2Rec_O.take(5)

[(u'user_id', (u'pub_domain', 0, 1)),
 (u'1410160', (u'1000', 1, 1)),
 (u'1157972', (u'1000', 0, 1)),
 (u'1061623', (u'1000', 1, 1)),
 (u'1169015', (u'1001', 1, 1))]

In [77]:
def red2list(a,b):
    if len(a)>1:
        c = a.append(b)
    else:
        
        c= a,b}

    return c

In [89]:
Log3Rec = Lorgs_norm.map(lambda tokens: (tokens[2],(tokens[1]))).cache()
Log2Rec_1 = Log3Rec.groupByKey().map(lambda (user, iterator): (user, [x for x in iterator],len(iterator)))
Log2Rec_1.count()

377465

In [92]:
Log2Rec_2 = Log2Rec_1.filter(lambda (x,y,z): z>1)

In [93]:
Log2Rec_2.count()

263422